<a href="https://colab.research.google.com/github/Arslonbekjon/Machine-learning-understanding-of-processes-and-analytical-approach/blob/main/Machine_Learning_prediction_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import sklearn


In [ ]:
URL="https://github.com/ageron/handson-ml2/blob/master/datasets/housing/housing.csv?raw=true"
df=pd.read_csv(URL)

In [ ]:
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(df, test_size=0.2, random_state=42)

X_train= train_set.drop("median_house_value", axis=1)
y= train_set['median_house_value'].copy()

X_num = X_train.drop('ocean_proximity',axis=1)

#Pipline quramiz

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
# Bizga kerakli ustunlar indekslari

rooms_ix, bedrooms_ix, population_ix, households_ix=3,4,5,6

class CombinedAttributesAdder(BaseEstimator,TransformerMixin):
  def __init__(self,add_bedrooms_per_room=True):
    self.add_bedrooms_per_room = add_bedrooms_per_room
  def fit(self,X,y=None):
    return self # bizni funksiyamiz faqat transformer. estimator emas
  def transform(self,X):
    rooms_per_household = X[:,rooms_ix]/X[:,households_ix]
    population_per_household=X[:,population_ix]/X[:,households_ix]
    if self.add_bedrooms_per_room:# add_bedrooms_per_room ustuni ixtiyoriy bo'ladi
       bedrooms_per_room=X[:,bedrooms_ix]/X[:,rooms_ix]
       return np.c_[X,rooms_per_household,population_per_household,bedrooms_per_room]
    else:
      return np.c_[X,rooms_per_household,population_per_household]


In [ ]:
from sklearn.pipeline import Pipeline

from sklearn.impute import SimpleImputer

from sklearn.preprocessing import OneHotEncoder, StandardScaler

num_pipeline = Pipeline([
    ('imputer',SimpleImputer(strategy='median')),
    ('attribs_adder', CombinedAttributesAdder(add_bedrooms_per_room=True)),
    ('std_scaler',StandardScaler())
])

In [ ]:
from sklearn.compose import ColumnTransformer

num_attribs=list(X_num)
cat_attribs = ['ocean_proximity']

full_pipeline = ColumnTransformer([
    ('num',num_pipeline,num_attribs),
    ('cat',OneHotEncoder(),cat_attribs)
])

In [ ]:
X_prepared = full_pipeline.fit_transform(X_train)

In [ ]:
X_prepared

In [ ]:
#Linear Regression

from sklearn.linear_model import LinearRegression

LR_model = LinearRegression()

In [ ]:
LR_model.fit(X_prepared,y)

In [99]:
test_data=X_train.sample(10)

In [100]:
X_train

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity
14196,-117.03,32.71,33.0,3126.0,627.0,2300.0,623.0,3.2596,NEAR OCEAN
8267,-118.16,33.77,49.0,3382.0,787.0,1314.0,756.0,3.8125,NEAR OCEAN
17445,-120.48,34.66,4.0,1897.0,331.0,915.0,336.0,4.1563,NEAR OCEAN
14265,-117.11,32.69,36.0,1421.0,367.0,1418.0,355.0,1.9425,NEAR OCEAN
2271,-119.80,36.78,43.0,2382.0,431.0,874.0,380.0,3.5542,INLAND
...,...,...,...,...,...,...,...,...,...
11284,-117.96,33.78,35.0,1330.0,201.0,658.0,217.0,6.3700,<1H OCEAN
11964,-117.43,34.02,33.0,3084.0,570.0,1753.0,449.0,3.0500,INLAND
5390,-118.38,34.03,36.0,2101.0,569.0,1756.0,527.0,2.9344,<1H OCEAN
860,-121.96,37.58,15.0,3575.0,597.0,1777.0,559.0,5.7192,<1H OCEAN


In [101]:
test_label = y.loc[test_data.index]
test_label


,median_house_value
5100,142600.0
13217,179700.0
16591,208100.0
1755,210800.0
12587,191900.0
518,398600.0
5056,118800.0
4654,257800.0
9094,218400.0
16457,154800.0


In [102]:
test_data_prepared = full_pipeline.transform(test_data)

predicted_labels = LR_model.predict(test_data_prepared)

In [103]:
predicted_labels

array([231464.41624031, 197740.59113009, 192077.11120708, 302928.70707788,
       171764.09916803, 337544.37760385, 181531.00620309, 207544.33487663,
       175701.6091117 , 192411.97912228])

In [104]:
pd.DataFrame({'Bashorat':predicted_labels, 'Asl qiymat':test_label})

,Bashorat,Asl qiymat
5100,231464.416240,142600.0
13217,197740.591130,179700.0
16591,192077.111207,208100.0
1755,302928.707078,210800.0
12587,171764.099168,191900.0
518,337544.377604,398600.0
5056,181531.006203,118800.0
4654,207544.334877,257800.0
9094,175701.609112,218400.0
16457,192411.979122,154800.0


#Next step is testing the model

In [105]:
test_set

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
20046,-119.01,36.06,25.0,1505.0,NaN,1392.0,359.0,1.6812,47700.0,INLAND
3024,-119.46,35.14,30.0,2943.0,NaN,1565.0,584.0,2.5313,45800.0,INLAND
15663,-122.44,37.80,52.0,3830.0,NaN,1310.0,963.0,3.4801,500001.0,NEAR BAY
20484,-118.72,34.28,17.0,3051.0,NaN,1705.0,495.0,5.7376,218600.0,<1H OCEAN
9814,-121.93,36.62,34.0,2351.0,NaN,1063.0,428.0,3.7250,278000.0,NEAR OCEAN
...,...,...,...,...,...,...,...,...,...,...
15362,-117.22,33.36,16.0,3165.0,482.0,1351.0,452.0,4.6050,263300.0,<1H OCEAN
16623,-120.83,35.36,28.0,4323.0,886.0,1650.0,705.0,2.7266,266800.0,NEAR OCEAN
18086,-122.05,37.31,25.0,4111.0,538.0,1585.0,568.0,9.2298,500001.0,<1H OCEAN
2144,-119.76,36.77,36.0,2507.0,466.0,1227.0,474.0,2.7850,72300.0,INLAND


In [106]:
X_test=test_set.drop('median_house_value', axis=1)
X_test

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity
20046,-119.01,36.06,25.0,1505.0,NaN,1392.0,359.0,1.6812,INLAND
3024,-119.46,35.14,30.0,2943.0,NaN,1565.0,584.0,2.5313,INLAND
15663,-122.44,37.80,52.0,3830.0,NaN,1310.0,963.0,3.4801,NEAR BAY
20484,-118.72,34.28,17.0,3051.0,NaN,1705.0,495.0,5.7376,<1H OCEAN
9814,-121.93,36.62,34.0,2351.0,NaN,1063.0,428.0,3.7250,NEAR OCEAN
...,...,...,...,...,...,...,...,...,...
15362,-117.22,33.36,16.0,3165.0,482.0,1351.0,452.0,4.6050,<1H OCEAN
16623,-120.83,35.36,28.0,4323.0,886.0,1650.0,705.0,2.7266,NEAR OCEAN
18086,-122.05,37.31,25.0,4111.0,538.0,1585.0,568.0,9.2298,<1H OCEAN
2144,-119.76,36.77,36.0,2507.0,466.0,1227.0,474.0,2.7850,INLAND


In [107]:
y_test=test_set['median_house_value'].copy()
y_test


,median_house_value
20046,47700.0
3024,45800.0
15663,500001.0
20484,218600.0
9814,278000.0
...,...
15362,263300.0
16623,266800.0
18086,500001.0
2144,72300.0


In [108]:
X_test_prepared = full_pipeline.transform(X_test)

In [109]:
y_predicted = LR_model.predict(X_test_prepared)

In [110]:
y_predicted

array([ 61874.25460143, 121853.52511139, 267770.94368091, ...,
       447837.04647878, 117275.9214608 , 185597.46125194])

In [111]:
from sklearn.metrics import mean_absolute_error

MAE=mean_absolute_error(y_test,y_predicted)

print("MAE=", MAE)

MAE= 50898.73953494079


In [112]:
from sklearn.metrics import mean_squared_error
MSE = mean_squared_error(y_test,y_predicted)
print("RMSE=",np.sqrt(MSE))

RMSE= 72701.32600762135


#Random Forest

In [113]:
from sklearn.ensemble import RandomForestRegressor

RF_model = RandomForestRegressor()
RF_model.fit(X_prepared,y)

RandomForestRegressor()

In [114]:
y_predicted = RF_model.predict(X_test_prepared)

In [115]:
from sklearn.metrics import mean_squared_error
MSE = mean_squared_error(y_test,y_predicted)
print("RMSE=",np.sqrt(MSE))

RMSE= 50136.97412981278


#Cross-Validation

In [116]:
X = df.drop("median_house_value",axis=1)
y = df['median_house_value'].copy()

X_prepared = full_pipeline.transform(X)

In [117]:
from sklearn.model_selection import cross_val_score

mse_scores=cross_val_score(LR_model, X_prepared,y,scoring="neg_mean_squared_error",cv=5)

In [118]:
def display_scores(scores):
  print("Scores:", scores)
  print("Mean:", scores.mean())
  print("Std.dev:", scores.std())

In [119]:
display_scores(np.sqrt(-mse_scores))

Scores: [73394.92502922 74814.24096819 75431.93119241 76608.78768825
 66196.48128669]
Mean: 73289.27323295095
Std.dev: 3694.71367872237


In [120]:
scores = cross_val_score(RF_model, X_prepared, y, scoring="neg_mean_squared_error", cv=10)
LR_rmse_scores = np.sqrt(-scores)
display_scores(LR_rmse_scores)

Scores: [98089.88206668 47566.39048711 65319.65751137 56722.8388208
 61247.60639075 59962.22848077 47197.27071743 79337.15717217
 74196.75315867 49406.1870426 ]
Mean: 63904.5971848352
Std.dev: 15313.992481549853


#Pickle

In [121]:
import pickle
filename = 'RF_model_pkl'  #Faylga istalgan nom beramiz
with open(filename,'wb') as file:
  pickle.dump(RF_model,file)

In [122]:
with open(filename,'rb') as file:
  model = pickle.load(file)

#Joblib

In [123]:
import joblib

filename = 'LR_model.jbl' #Faylga istalgan nom beramiz
joblib.dump(LR_model, filename)

['LR_model.jbl']

In [124]:
model = joblib.load(filename)

In [125]:
scores = cross_val_score(model, X_prepared,y, scoring='neg_mean_squared_error',cv=5)
LR_rmse_scores = np.sqrt(-scores)
display_scores(LR_rmse_scores)

Scores: [73394.92502922 74814.24096819 75431.93119241 76608.78768825
 66196.48128669]
Mean: 73289.27323295095
Std.dev: 3694.71367872237
